In [ ]:
from datetime import datetime

from pymongo import MongoClient
import pandas as pd

import constants
from tqdm import tqdm


In [ ]:
mongo = MongoClient(constants.mongo_conn_string)
mongo_lt_col = mongo[constants.mongo_db][constants.mongo_lt_annotated_col]

In [ ]:
query = {}

cursor = mongo_lt_col.find(query, {"_id":0, "text":0})

data = [i for i in cursor]
print(len(data))

df = pd.DataFrame(data)
print(df.shape)
df.head()

In [ ]:
#df.to_parquet("Data.parquet")

# Basic Data

In [ ]:
df.year.value_counts()

In [ ]:
df.cabinet.value_counts()

In [ ]:
df.president.value_counts()

# Adding dummies for media & presidential attention for each policy sphere

In [ ]:
ministries = set()
for i in list(df.cabinet_ents):
    for a in i:
        ministries.add(a)
    
for item in tqdm(list(ministries)):
    df[item]= df.apply(lambda x : int(item in x["cabinet_ents"]), axis = 1)


def prez_attention( list_min , minister):
    if "Prezidentas" in list_min and minister in list_min:
        return 1
    else:
        return 0
    
for item in tqdm(list(ministries)):
    if item != "Prezidentas":
        df["president_"+item]= df.apply(lambda x : prez_attention(x["cabinet_ents"], item), axis = 1)
    
df.head() 

In [ ]:
df.columns

In [ ]:
df.to_parquet("DataEnhanced.parquet")

In [ ]:
df.to_csv("DataEnhanced.csv")